# Getting Started with TensorFlow

This notebook contains some small code pieces to give you a sense of basic tensorflow functions. You can use this as a space to play around with tensorflow functionality.

In [1]:
import tensorflow as tf
import numpy as np

## Session

The code below demonstrates how to use `sess.run()` to make tensorflow talk to python and vice versa.

In [2]:
sess = tf.Session()

# an example with strings
hello_tensorflow = tf.constant('hello world!')
print(hello_tensorflow)

hello_python = sess.run(hello_tensorflow)
print(hello_python), print('-'*50)

# an example with numbers
tensorflow_vector = tf.constant([1, 2, 3, 4])
print(tensorflow_vector)

python_vector = sess.run(tensorflow_vector)
print(python_vector)

Tensor("Const:0", shape=(), dtype=string)
b'hello world!'
--------------------------------------------------
Tensor("Const_1:0", shape=(4,), dtype=int32)
[1 2 3 4]


## Placeholders

The code below demonstrates how to use `feed_dict` in conjunction with placeholder variables.

In [3]:
# make two placeholders
foo = tf.placeholder(tf.float32, shape=(2,))
bar = tf.placeholder(tf.float32, shape=(2,))

# computational graph
baz = foo * bar

# values for the placeholders
feed_dict = {
    foo: np.array([1, 2]),
    bar: np.array([2, 3])
}

sess.run(baz, feed_dict=feed_dict)

array([ 2.,  6.], dtype=float32)

## Computing Gradients

You can use [`tf.gradients(y, x)`](https://www.tensorflow.org/api_docs/python/tf/gradients) to compute the derivative of a variable `y` with respect to `x`.

In [4]:
x1 = tf.placeholder(tf.float32)
w1 = tf.placeholder(tf.float32)
w2 = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

x2 = x1 * w1
x3 = x2 * w2

loss = 0.5 * (y - x3)**2

feed_dict = {
    x1: 1,
    w1: 1,
    w2: 1,
    y: 0
}

for variable, name in zip([x1, w1, w2, y], ['x1', 'w1', 'w2', 'y']):
    val = sess.run(tf.gradients(loss, variable), feed_dict=feed_dict)[0]
    print('deriv of {} w.r.t. loss = {}'.format(name, val))

deriv of x1 w.r.t. loss = 1.0
deriv of w1 w.r.t. loss = 1.0
deriv of w2 w.r.t. loss = 1.0
deriv of y w.r.t. loss = -1.0


Of course, the whole point is that `tensorflow` makes it easy to scale up the above example to large matrix multiplications and nonlinear functions. Here is a very simple three layer feedforward network.

In [13]:
# weights between layers 1 -> 2 and layers 2 -> 3
w1 = tf.Variable(np.random.randn(100, 200), dtype=tf.float32)
w2 = tf.Variable(np.random.randn(200, 10), dtype=tf.float32)

# activation of each layer
x1 = tf.placeholder(tf.float32, shape=(None, 100))
x2 = tf.nn.tanh(tf.matmul(x1, w1))
x3 = tf.matmul(x2, w2)

# Note the shape of `x` has a "None" in it. This tells tensorflow that
# the shape of that dimension will be determined at runtime, which can be
# a handy trick when you aren't sure what the dimensions of your data are.

# loss function
target = tf.placeholder(tf.float32, shape=10)
loss = tf.nn.l2_loss(x3 - target)

# initialize all variables
sess.run(tf.global_variables_initializer())

# define the input and target for the network. You can try changing these
# values if you like.
feed_dict = {
    x1: np.random.randn(1, 100)*.1,
    target: np.random.randn(10)
}

# compute gradients
for variable, name in zip([w1, w2], ['w1', 'w2']):
    val = sess.run(tf.gradients(loss, variable), feed_dict=feed_dict)[0]
    print('deriv of {} w.r.t. loss = {}'.format(name, val))

deriv of w1 w.r.t. loss = [[ 0.56114584  1.70765328  0.43764409 ...,  1.54541659  0.04804779
   0.08327151]
 [ 0.68632495  2.08859253  0.53527272 ...,  1.89016449  0.05876618
   0.10184753]
 [-2.98942351 -9.09727669 -2.33148575 ..., -8.23298359 -0.25596768
  -0.44361699]
 ..., 
 [ 2.41859221  7.36014938  1.88628793 ...,  6.66089344  0.20709059
   0.35890821]
 [ 1.60476017  4.88353252  1.25157094 ...,  4.41956949  0.13740668
   0.23813918]
 [-1.64216673 -4.99736643 -1.28074467 ..., -4.52258825 -0.14060959
  -0.24369015]]
deriv of w2 w.r.t. loss = [[  0.2368754   -0.02027308  -0.02844641 ...,  -0.1672783   -0.13502629
    0.20465143]
 [ -0.96353322   0.08246438   0.11571088 ...,   0.68043453   0.54924363
   -0.83245647]
 [ -4.83959532   0.41419876   0.5811879  ...,   3.41765881   2.75871873
   -4.18122864]
 ..., 
 [  7.24715424  -0.62025064  -0.87031204 ...,  -5.11784554  -4.13110161
    6.26126862]
 [-15.03609371   1.28687024   1.80568707 ...,  10.61829185   8.57103729
  -12.99061871]
 